# $\color{teal}{\text{4.2 Iteratively Reweighted Least Squares}}$
Due to the sparsity favoring property of the l1-norm and some other assumptions, the l1-minimization can be used as an approximation of the P0 problem in CS. One algorithm to solve the P1 problem is called iteratively reweighted least squares (IRLS) algorithm that iteratively solves weigthed l2-minimization problems. Unfortunately, the IRLS is just a proxy and the convergence to the l1-minimizer is not guaranteed. Nevertheless, if the measurement matrix A satisfies the (stable) NSP, the IRLS recovers every s-sparse vector from y=AX, and provides similar error estimates as the l1-minimizer.  

We observe that $|t| = \frac{|t|^2}{|t|}$ for all $t \neq 0$. By this trivial reformulation, we can recast the l1-minimization as a weighted least squares problem. Assume that for $A \in \mathbb{C}^{m \times N}$ with $m \leq N$ we have that $x^{\#}$ is a minimizer of

$$ \underset{x \in \mathbb{C}^{N}}{\mathrm{min}} ||x||_1 \;\;\; \text{subject to } Ax=y.$$

If $x^{\#}_j \neq 0$ for all $j \in [N]$, then it also minimizer the weigthed l2-problem given as

$$ \underset{x \in \mathbb{C}^{N}}{\mathrm{min}} \sum_{j=1}^{N} \frac{|x_j|^{2}}{|x^{\#}_j|} \;\;\; \text{subject to } Ax=y.$$

This reformulation simplifies our calculations, since minimizing the smooth quadratic function $|t|^{2}$ is easier than minimizing the nonsmooth function $|t|$. But this comes with a cost: we do not know $x^{\#}$ in advance, neither can we expect that $x^{\#}_j \neq 0$ for all $j \in [N]$, since we target sparse solutions. If the l1-minimizer $x^{\#} \in \mathbb{R}^{N}$ is unique, then it is $m$-sparse. We further introduce the functional 

$$\mathcal{J}(x,w,\epsilon) = \frac{1}{2}\left[\sum_{j=1}^{N}|x_j|^{2}w_j + \sum_{j=1}^{N}(\epsilon^2w_j + \frac{1}{w_j})\right]$$

with $x \in \mathbb{C}^{N}, \epsilon \geq 0$ and $w \in \mathbb{R}^{N}_{+}$. We denote by $(x^{(n)})^{*}$ the nonincreasing rearrangement of the iterate $x^{(n)}$.

In [85]:
import numpy as np
import random
import math

#omp
m = 300
N = 1000
s = 30
gamma = 1/(2*N)

#ground truth
x_true = np.zeros(N)
indices = random.sample(range(N),s) #randomly sample s out of 1000 indices
x_true[indices] = np.random.rand(s)

#random measurement matrix
A = np.random.randn(m,N)
A = A/math.sqrt(m)
#simulated measurements
y = A @ x_true

In [86]:
def restrict_on_support(matrix,support_set):
    """Restrict matrix on support set by copying all elements in the columns indicated by 
    support_set and setting the other ones to zero."""
    
    [rows,clms] = matrix.shape
    matrix_supp = np.zeros((rows,clms), dtype='float32')
    matrix_supp[:,support_set] = matrix[:,support_set]
    return matrix_supp

In [87]:
def rearrange_non_inc(vector):
    vector_ = np.sort(vector)
    vector_non_inc = vector_[::-1]
    return vector_non_inc

In [88]:
def irls(A,y,gamma,s, iters):
    #initialization:
    w_0 = np.ones(N)
    epsilon_0 = 1
    
    #IRLS1:
    for i in range(iters):
        if i==0:
            w_0_sqrt = np.array([math.sqrt(w) for w in w_0])
            D_sqrt = np.diag(w_0_sqrt)
            D_sqrt_inv = np.linalg.inv(D_sqrt)
            epsilon = epsilon_0
        
        x_new = D_sqrt_inv@np.linalg.pinv(A@D_sqrt_inv)@y
        epsilon_new = min(epsilon, gamma*rearrange_non_inc(x_new)[s])
        w_new = np.array([math.sqrt(np.absolute(x_j)**2+epsilon_new**2) for x_j in x_new])
        
        epsilon = epsilon_new
        
        w_sqrt = np.array([math.sqrt(w) for w in w_new])
        D_sqrt = np.diag(w_sqrt)
        D_sqrt_inv = np.linalg.inv(D_sqrt) 
        
        err = np.linalg.norm(A@x_new-y,2) #l2-error
        print(err)
        
    return x_new
        

In [89]:
n_iters = 100
x_hat = irls(A,y,gamma,s,n_iters)

6.690276343932392e-15
1.891850412693194e-14
7.458180063323482e-15
1.08438910405026e-14
7.422411672947018e-15
1.0418250227924591e-14
7.527609659004484e-15
1.2974521875780226e-14
6.90474826949231e-15
1.2925955316186398e-14
8.370568587257576e-15
1.2307402294249415e-14
7.528886795876672e-15
1.1245400025240368e-14
8.32144320035038e-15
1.1932747701187256e-14
8.228712921147796e-15
1.1006493327021882e-14
9.520455332191001e-15
1.0819027938058294e-14
8.43372641779954e-15
1.2025511534293928e-14
9.122003316772356e-15
1.0827515977843024e-14
8.158238567340206e-15
1.1549316390981457e-14
7.167948982812606e-15
1.2423672438608433e-14
8.040618138314116e-15
1.2471533085232877e-14
7.910038511511248e-15
1.216303601157606e-14
8.153329748280716e-15
1.1541753045539637e-14
9.581004531879411e-15
1.3873626775008396e-14
9.380310719472599e-15
1.2615128579860495e-14
8.10261481529658e-15
1.2496561069687757e-14
8.589555811998676e-15
1.4027065954348477e-14
1.0072277946506053e-14
1.0987266737232566e-14
8.868116502689092

In [84]:
err = np.linalg.norm(x_hat-x_true,2) #l2-error
print(err)

1.389000953686155e-09
